In [ ]:
# Common imports
import numpy as np
import os
import pandas as pd
import tensorflow as tf

# Get the data

In [ ]:
housing = pd.read_csv("housing.csv")
housing.head()

In [ ]:
#Drop the missing values
housing.dropna(axis=0, inplace=True)

# Let's also reset the index
housing.reset_index(inplace=True, drop=True)


housing.describe()

# Prepare the data for Machine Learning algorithms

In [ ]:
#Set the training and test data sets
housing_num = housing.drop("ocean_proximity", axis=1) # drop labels 

#Select the label
housing_target = housing[["ocean_proximity"]]

### Standardize the data

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

housing_num_std = scaler.fit_transform(housing_num)

In [ ]:
housing_num_std

In [ ]:
housing_num_std.shape

### Create the label column

Tensorflow wants the labels in integer form. So, we need to do Ordinal Encoding, then convert the numbers to integers.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()

housing_labels_ord = ordinal_encoder.fit_transform(housing_target)

housing_labels_ord[:10]

In [ ]:
housing_target[:10]

In [ ]:
# Data type is float. It needs to be integer
housing_labels_ord.dtype

In [ ]:
#Convert to integer

housing_labels_int = housing_labels_ord.astype(int)

housing_labels_int.dtype

In [ ]:
housing_labels_int.shape

# Split data (train/test)

In [ ]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(housing_num_std, housing_labels_int, test_size=0.3)

# Multiclass classification using Keras



In [ ]:
from tensorflow import keras

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

# fix random seed for reproducibility
np.random.seed(42)

In [ ]:
#Define the model: for multi-class

model = Sequential()

model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(10, activation='relu'))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model.add(Dense(5, activation='softmax'))


In [ ]:
# Compile model

#Optimizer:
sgd = keras.optimizers.SGD(lr=0.05)


#You need to use "categorical_crossentropy" for mutli-class
#but since our target is ordinal, we need to use "sparse_..."
#if it is binary classification, then use binary_crossentropy

model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# Fit the model

model.fit(train_x, train_y, epochs=100, batch_size=100)

In [ ]:
# evaluate the model

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


# Optimizers, Learning rate, Dropout, Initialization & Activation functions

In [ ]:
#Define the model: for multi-class


#Set the learning rate:
lr=0.001


#Available optimizers:
adagrad = keras.optimizers.Adagrad(lr=lr, decay=0.0)
sgd = keras.optimizers.SGD(lr=lr, momentum=0.0, decay=0.0, nesterov=False)
rmsprop = keras.optimizers.RMSprop(lr=lr, rho=0.9, decay=0.0)
adam = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)
nesterov_adam = keras.optimizers.Nadam(lr=lr, beta_1=0.9, beta_2=0.999, schedule_decay=0.004)

#Initializations:
xavier = keras.initializers.glorot_normal(seed=None)
he = keras.initializers.he_normal(seed=None)


# Activation functions. Uncomment only one
#activation = 'elu' 
activation = 'relu'
#activation = 'tanh'
#activation = 'sigmoid'



#See the droput layers below:
model = Sequential()

model.add(Dense(50, input_dim=9, activation=activation, kernel_initializer=xavier))

model.add(Dropout(0.2))

model.add(Dense(25, activation=activation, kernel_initializer=xavier))

model.add(Dropout(0.2))

model.add(Dense(10, activation=activation, kernel_initializer=xavier))

#final layer: there has to be 5 nodes with softmax (because we have 5 categories)
model.add(Dense(5, activation='softmax'))



#Compile"
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=nesterov_adam, metrics=['accuracy'])

In [ ]:
# Fit the model
model.fit(train_x, train_y, epochs=100, batch_size=100)

In [ ]:

# evaluate the model
scores = model.evaluate(test_x, test_y)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# Early stopping based on validation results

To do this, you need to send the validation data sets to the fit() function and use a callback.

EarlyStopping Arguments:

**monitor:** quantity to be monitored.<br>
**min_delta:** minimum change in the monitored quantity to qualify as an improvement, i.e. an absolute change of less than min_delta, will count as no improvement.<br>
**patience:** number of epochs with no improvement after which training will be stopped.<br>
**verbose:** verbosity mode.<br>
**mode:** one of {auto, min, max}. In min mode, training will stop when the quantity monitored has stopped decreasing; in max mode it will stop when the quantity monitored has stopped increasing; in auto mode, the direction is automatically inferred from the name of the monitored quantity.<br>
**baseline:** Baseline value for the monitored quantity to reach. Training will stop if the model doesn't show improvement over the baseline.<br>
**restore_best_weights:** whether to restore model weights from the epoch with the best value of the monitored quantity. If False, the model weights obtained at the last step of training are used.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

callback = [earlystop]

model.fit(train_x, train_y, validation_data=(test_x, test_y), 
          epochs=100, batch_size=100, callbacks=callback)

In [ ]:
scores = model.evaluate(test_x, test_y)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))